In [20]:
import pandas as pd

# Load marketing data
marketing_data = pd.read_csv('data/de_hw_marketing_data.csv')

# Load sales outreach data
sales_data = pd.read_csv('data/de_hw_sales_outreach_data.csv')

# Load contact data
contact_data = pd.read_excel('data/de_hw_contact_data.xlsx')

# Load opportunity data
opportunity_data = pd.read_csv('data/de_hw_opportunity_data.csv')

# Rename columns to maintain consistency
opportunity_data = opportunity_data.rename(columns={
 'account_id': 'AccountId', 
 'opportunity_id': 'OpportunityId', 
 'pipeline_amount': 'PipelineAmount', 
 'Opportunity_Created_Date': 'OpportunityCreatedDate', 
 'sales_segment': 'SalesSegment'
})

contact_data = contact_data.rename(columns={
 'contact_id': 'ContactId', 
 'account_id': 'AccountId'
})

marketing_data = marketing_data.rename(columns={
 'contact_id': 'ContactId',
 'marketing_touchpoint_id': 'TouchpointId', 
 'marketing_touchpoint_date': 'TouchpointDate',
 'channel_name': 'ChannelName'
})

sales_data = sales_data.rename(columns={
 'contact_id': 'ContactId',
 'sales_touchpoint_id': 'TouchpointId', 
 'sales_touchpoint_date': 'TouchpointDate',
 'channel_name': 'ChannelName'
})

# Merge contact data with marketing and sales data
marketing_data = marketing_data.merge(contact_data, on='ContactId')
sales_data = sales_data.merge(contact_data, on='ContactId')

# Combine marketing and sales data into a single dataframe
marketing_data['TouchpointType'] = 'Marketing'
sales_data['TouchpointType'] = 'Sales'

all_touchpoints = pd.concat([marketing_data, sales_data], ignore_index=True)

# Clean up the AccountId columns after merging
all_touchpoints = all_touchpoints.loc[:,~all_touchpoints.columns.duplicated()]

# Merge touchpoints with opportunity data
all_touchpoints = all_touchpoints.merge(opportunity_data, on='AccountId')

# Convert date columns to datetime
all_touchpoints['TouchpointDate'] = pd.to_datetime(all_touchpoints['TouchpointDate'])
all_touchpoints['OpportunityCreatedDate'] = pd.to_datetime(all_touchpoints['OpportunityCreatedDate'])

# Filter touchpoints within the 90-day window
all_touchpoints['DaysBeforeOpportunity'] = (all_touchpoints['OpportunityCreatedDate'] - all_touchpoints['TouchpointDate']).dt.days
valid_touchpoints = all_touchpoints[(all_touchpoints['DaysBeforeOpportunity'] >= 0) & (all_touchpoints['DaysBeforeOpportunity'] <= 90)]

# Sort touchpoints by Opportunity ID and Days Before Opportunity
valid_touchpoints = valid_touchpoints.sort_values(by=['OpportunityId', 'DaysBeforeOpportunity'])

# Drop duplicates to keep only the first touchpoint
first_touchpoints = valid_touchpoints.drop_duplicates(subset=['OpportunityId'], keep='first')

# Assign pipeline amount to the first touchpoint
first_touchpoints = first_touchpoints[['OpportunityId', 'TouchpointId', 'ChannelName', 'PipelineAmount', 'SalesSegment']]

assert len(first_touchpoints) > 0

all_touchpoints.to_csv('all_touchpoints.csv')

first_touchpoints.head()

,OpportunityId,TouchpointId,ChannelName,PipelineAmount,SalesSegment
10806,DE64p00000Uq2cuAABHW,SAT4p00001g2Ot5EAEID,Outbound,13124.254780,Commercial
13179,DE64p00000Uq2oMAARHW,SAT4p00001g2PYoEAMID,Outbound,4518.917197,Commercial
8520,DE64p00000Uq2pZAARHW,SAT4p00001eoxyWEAQID,Outbound,7005.095541,Commercial
6056,DE64p00000Uq35SAARHW,SAT4p00001g2XYHEA2ID,Outbound,15774.267520,Commercial
3003,DE64p00000Uq3A8AAJHW,MKv4p00000ifKF7AAMID,Adwords,12760.509550,Commercial


In [18]:
# Group by ChannelName and SalesSegment to calculate the total pipeline
channel_pipeline = first_touchpoints.groupby(['ChannelName', 'SalesSegment'])['PipelineAmount'].sum().reset_index()

# Find the channel that sourced the most pipeline
total_pipeline_by_channel = channel_pipeline.groupby('ChannelName')['PipelineAmount'].sum().reset_index()
top_channel = total_pipeline_by_channel.sort_values(by='PipelineAmount', ascending=False).iloc[0]

print(f'The channel {top_channel.ChannelName} has the highest pipeline amount, with a total of {top_channel.PipelineAmount}')

channel_pipeline

The channel Outbound has the highest pipeline amount, with a total of 34957059.91800094


,ChannelName,SalesSegment,PipelineAmount
0,Adwords,Commercial,1.503281e+06
1,Adwords,Enterprise,1.813726e+06
2,Adwords,Mid Market,7.006464e+05
3,Event,Commercial,6.599764e+04
4,Event,Enterprise,4.207697e+05
5,Event,Mid Market,1.390357e+05
6,Outbound,Commercial,9.049766e+06
7,Outbound,Enterprise,2.163466e+07
8,Outbound,Mid Market,4.272637e+06
9,Webinar,Commercial,2.983480e+05
